In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from mpl_toolkits.mplot3d import Axes3D

import datetime
import seaborn as sns

import time

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier,RandomForestClassifier,AdaBoostClassifier, IsolationForest
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC

from sklearn.model_selection import cross_val_predict,cross_val_score,train_test_split
from sklearn.metrics import classification_report,confusion_matrix,roc_curve,auc,precision_recall_curve

import os

%matplotlib inline
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

In [ ]:
df = pd.read_csv("../input/creditcard.csv")


In [ ]:
## add time stamps
df['hour'] = df['Time'].apply(lambda x: np.ceil(float(x)/3600) % 24)
## rescale amount
## data pre processing
df["Normalized Amount"] = (df["Amount"]-np.mean(df["Amount"]))/np.std(df["Amount"])
df.drop(["Time","Amount"],axis=1,inplace=True)

In [ ]:
def oversample(df_x,frauds,nonfrauds,percent):
    n_nonfrauds = len(nonfrauds)
    oversample_frauds_indices = np.array(np.random.choice(frauds,round(percent*n_nonfrauds),replace =True))
    oversample_indices = np.concatenate([np.array(nonfrauds),oversample_frauds_indices])
    print("We have", round(percent*n_nonfrauds)," fraud cases" )
    print("We have", (n_nonfrauds) ,"non fraud cases" )
    oversample = df_x.iloc[oversample_indices]
    return oversample 

In [ ]:
def sample_split(df,times):
    df_label = df.Class
    df_feature = df.drop("Class", axis= 1)
    data_features_train,data_features_test,data_labels_train,data_labels_test=train_test_split(df_feature, df_label,test_size=0.3)
    df_train = np.concatenate( [data_features_train,data_labels_train])
    frauds = np.array(df[df.Class == 1].index)
    nonfrauds = np.array(df[df.Class == 0].index)
    
    sample_data = oversample(df,frauds,nonfrauds,times)
    sample_label = sample_data.Class
    sample_feature = sample_data.drop("Class", axis= 1)
    
    sample_features_train,sample_features_test,sample_labels_train,sample_labels_test = train_test_split(sample_feature,sample_label,test_size=0.3)
    return [sample_features_train,sample_features_test,sample_labels_train,sample_labels_test,data_features_train,data_features_test,data_labels_train,data_labels_test]
def choose_supervised_model(model):
    if model == "LR":
        print("Logsitic regression model")   
        clf=LogisticRegression()
    elif model == "SVC":
        print("SVC model")   
        clf = LinearSVC()
    elif model == "RF":
        print("Random Forest model")   
        clf = RandomForestClassifier(n_estimators=100)
    return clf
def report_oversample(df,model):
    for i in [0.01,0.5,1.00,1.50,1.80,2.0]:
        print("ratio", i)
        print()
        sample_features_train,sample_features_test,sample_labels_train,sample_labels_test,data_features_train,data_features_test,data_labels_train,data_labels_test= sample_split(df,i)
        clf = choose_supervised_model(model)
        clf.fit(sample_features_train,sample_labels_train)
        pred=clf.predict(data_features_test)
        print() 
        print(classification_report(data_labels_test,pred))
        print("======================================")

In [ ]:
report_oversample(df,"RF")

In [ ]:
df_label = df.Class
df_feature = df.drop("Class", axis= 1)
data_features_train,data_features_test,data_labels_train,data_labels_test=train_test_split(df_feature, df_label,test_size=0.3)
df_train = np.concatenate( [data_features_train,data_labels_train])
frauds = np.array(df[df.Class == 1].index)
nonfrauds = np.array(df[df.Class == 0].index)